In [1]:
from glob import glob
import ast
import pandas as pd

def clean_data(df):
    # Explode 'fsq_category_ids' and duplicate 'name' for each ID
    # Ensure 'fsq_category_ids' is a list
    df['fsq_category_ids'] = df['fsq_category_ids'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    # Explode the DataFrame to create a new row for each ID
    df = df.explode('fsq_category_ids')
    # Filter rows where 'fsq_category_ids' is not null
    df = df[df['fsq_category_ids'].notna()]
    # Count occurrences of each name within the same ID and add a column
    df['Count'] = df.groupby(['fsq_category_ids', 'name'])['name'].transform('count')
    # Group by 'fsq_category_ids', sort 'name' in each group by 'Count' in descending order, keep top 10 in each group, and remove duplicate names
    df = (
        df.sort_values(by=['fsq_category_ids', 'Count'], ascending=[True, False])
          .drop_duplicates(subset=['fsq_category_ids', 'name'])
          .groupby('fsq_category_ids')
          .head(100)
    )
    # Sort categories by count in descending order
    df = df.sort_values(by="Count", ascending=False)
    return df

files = glob('data/places-*.zstd.parquet')
df = pd.concat([pd.read_parquet(f)[["name", "fsq_category_ids"]] for f in files[:5]], ignore_index=True)

df_clean = clean_data(df.copy())
df_clean.head()

,name,fsq_category_ids,Count
73980,Western Union,63be6904847c3692a84b9b3d,5993
83856,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032
34,Starbucks,4bf58dd8d48988d1e0931735,4893
15911,Blue Rhino,63be6904847c3692a84b9b34,4818
74113,Redbox,4bf58dd8d48988d126951735,4718


In [2]:
df_clean

,name,fsq_category_ids,Count
73980,Western Union,63be6904847c3692a84b9b3d,5993
83856,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032
34,Starbucks,4bf58dd8d48988d1e0931735,4893
15911,Blue Rhino,63be6904847c3692a84b9b34,4818
74113,Redbox,4bf58dd8d48988d126951735,4718
...,...,...,...
162907,Davis Community Transit,63be6904847c3692a84b9c2d,1
161454,Unitrans,63be6904847c3692a84b9c2d,1
159563,Michael's Transportaion,63be6904847c3692a84b9c2d,1
300744,24 Divisadero Bus,63be6904847c3692a84b9c2d,1


In [3]:
df_clean.to_csv('business_names_new.csv')